In [55]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets


In [56]:
# ===============================
# MODEL CONFIGURATION SYSTEM
# ===============================

# adjust the model inputs for the stochastic simulation based on your needs
# assumeed parallelism is 8 by default
# costs are in $ per million tokens
class ModelConfig:
    """Configuration for a specific model with its unique parameters."""
    def __init__(self, name, c_in, c_out, t_in, t_out,
                 mu_Lin, sigma_Lin, mu_Lout, sigma_Lout,
                 acc_mean, acc_std, default_parallel=8):
        self.name = name
        self.c_in = c_in      # $ per input token
        self.c_out = c_out    # $ per output token
        self.t_in = t_in      # sec per input token
        self.t_out = t_out    # sec per output token
        self.mu_Lin = mu_Lin
        self.sigma_Lin = sigma_Lin
        self.mu_Lout = mu_Lout
        self.sigma_Lout = sigma_Lout
        self.acc_mean = acc_mean
        self.acc_std = acc_std
        self.default_parallel = default_parallel

    def __str__(self):
        return (f"{self.name}: "
                f"Cost(${self.c_in*1e6:.2f}/${self.c_out*1e6:.2f}/M), "
                f"ACC({self.acc_mean:.3f}±{self.acc_std:.3f}), "
                f"P={self.default_parallel}")

# Predefined models (params chosen to make k≈10–20 optimal under defaults)
MODEL_CONFIGS = {
    "gpt5": ModelConfig(
        name="GPT-5",
        c_in=1.25 / 1_000_000,
        c_out=10.00 / 1_000_000,
        t_in=0.0005,     # ≈200 tok/s in
        t_out=0.005,     # ≈200 tok/s out
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.85, acc_std=0.08,
        default_parallel=8
    ),
    "gpt5-mini": ModelConfig(
        name="GPT-5 Mini",
        c_in=0.25 / 1_000_000,
        c_out=2.00 / 1_000_000,
        t_in=0.00025, t_out=0.002,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.83, acc_std=0.09,
        default_parallel=8
    ),
    "gpt5-nano": ModelConfig(
        name="GPT-5 Nano",
        c_in=0.05 / 1_000_000,
        c_out=0.40 / 1_000_000,
        t_in=0.00010, t_out=0.0010,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.80, acc_std=0.10,
        default_parallel=8
    ),
    "nvidia-nemotron-ultra-253b": ModelConfig(
        name="Nvidia Llama Nemotron Ultra 253B",
        c_in=0.90 / 1_000_000,
        c_out=2.80 / 1_000_000,
        t_in=0.0010, t_out=0.0100,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.84, acc_std=0.02,
        default_parallel=8
    ),
    "nvidia-nemotron-h-47b": ModelConfig(
        name="Nvidia Nemotron H 47B",
        c_in=0.40 / 1_000_000,
        c_out=1.50 / 1_000_000,
        t_in=0.0004, t_out=0.0040,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.81, acc_std=0.025,
        default_parallel=8
    ),
    "nvidia-nemotron-nano-9b-v2": ModelConfig(
        name="Nvidia Nemotron Nano 9B v2",
        c_in=0.20 / 1_000_000,
        c_out=1.00 / 1_000_000,
        t_in=0.00012, t_out=0.0012,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.79, acc_std=0.03,
        default_parallel=8
    ),
    "qwen3-max": ModelConfig(
        name="Qwen3-Max",
        c_in=0.90 / 1_000_000,
        c_out=2.40 / 1_000_000,
        t_in=0.0008, t_out=0.0080,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.835, acc_std=0.042,
        default_parallel=8
    ),
    "qwen3-next-80b-a3b": ModelConfig(
        name="Qwen3-Next-80B-A3B",
        c_in=0.50 / 1_000_000,
        c_out=1.25 / 1_000_000,
        t_in=0.0004, t_out=0.0040,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.805, acc_std=0.058,
        default_parallel=8
    ),
    "qwen3-30b-a3b": ModelConfig(
        name="Qwen3-30B-A3B",
        c_in=0.35 / 1_000_000,
        c_out=0.90 / 1_000_000,
        t_in=0.00025, t_out=0.0020,
        mu_Lin=1500, sigma_Lin=120,
        mu_Lout=3000, sigma_Lout=250,
        acc_mean=0.785, acc_std=0.08,
        default_parallel=8
    ),
}

# sample cost and latency constraints
C_max_default = 0.50   # $
T_max_default = 60.0   # seconds

In [57]:
# ===============================
# MONTE CARLO SIMULATION WITH AVERAGING
# ===============================

def simulate_mc_with_model(k, model_config, mc_trials=1000, parallel_factor=8, seed=42):
    """
    Monte Carlo simulation: run mc_trials independent trials and return statistics.
    This provides robust estimates with confidence intervals.
    """
    rng = np.random.default_rng(seed)
    
    costs, times, accs = [], [], []
    
    for trial in range(mc_trials):
        # Draw per-inference lengths for this trial
        Lin = rng.normal(model_config.mu_Lin, model_config.sigma_Lin, size=k)
        Lin = np.clip(Lin, 1, 16 * model_config.mu_Lin)
        Lout = rng.normal(model_config.mu_Lout, model_config.sigma_Lout, size=k)
        Lout = np.clip(Lout, 1, 16 * Lin)

        # Per-inference cost and time
        trial_costs = model_config.c_in * Lin + model_config.c_out * Lout
        trial_times = model_config.t_in * Lin + model_config.t_out * Lout

        # Total cost for this trial
        total_cost = trial_costs.sum()

        # Parallel-adjusted time for this trial
        if parallel_factor > 1:
            batch_count = int(np.ceil(k / parallel_factor))
            mean_time = float(np.mean(trial_times))
            total_time = batch_count * mean_time
        else:
            total_time = float(np.sum(trial_times))

        # Best-of-k accuracy for this trial
        trial_accs = rng.normal(model_config.acc_mean, model_config.acc_std, size=k)
        trial_accs = np.clip(trial_accs, 0.0, 1.0)
        best_acc = float(np.max(trial_accs))

        costs.append(total_cost)
        times.append(total_time)
        accs.append(best_acc)

    # Convert to numpy arrays for statistics
    costs = np.array(costs)
    times = np.array(times)
    accs = np.array(accs)

    # Calculate statistics with confidence intervals
    def calc_stats(data):
        return {
            "mean": float(np.mean(data)),
            "std": float(np.std(data)),
            "ci95": (float(np.percentile(data, 2.5)), float(np.percentile(data, 97.5)))
        }

    return {
        "cost": calc_stats(costs),
        "time": calc_stats(times),
        "acc": calc_stats(accs)
    }


In [58]:
# ===============================
# UPDATED CONSTRAINT LOGIC - PER INFERENCE BASIS
# ===============================

def find_feasible_k_mc(model_config, C_max_per_inference, T_max_per_inference, k_max=200, mc_trials=500, parallel_factor=8, seed=42):
    """Find feasible k values using per-inference constraints."""
    feasible = []
    print(f"Computing feasible region with per-inference constraints...")
    print(f"  C_max per inference: ${C_max_per_inference:.4f}")
    print(f"  T_max per inference: {T_max_per_inference:.3f}s")
    
    for k in range(1, k_max + 1):
        if k % 20 == 0:
            print(f"  k={k}/{k_max}")
            
        stats = simulate_mc_with_model(k, model_config, mc_trials, parallel_factor, seed + k)
        
        # Calculate per-inference costs and times
        cost_per_inference = stats["cost"]["mean"] / k
        time_per_inference = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        
        # Use per-inference values for feasibility check
        if cost_per_inference <= C_max_per_inference and time_per_inference <= T_max_per_inference:
            feasible.append((k, stats))
    
    return feasible


def find_accuracy_optimal_mc(model_config, C_max_per_inference, T_max_per_inference, acc_min=0.0, k_max=200, 
                            mc_trials=500, parallel_factor=8, seed=42):
    """Find accuracy-optimal k using per-inference constraints."""
    print(f"Finding accuracy-optimal solution with per-inference constraints...")
    print(f"  C_max per inference: ${C_max_per_inference:.4f}")
    print(f"  T_max per inference: {T_max_per_inference:.3f}s")
    
    best = None
    best_acc = -1
    
    for k in range(1, k_max + 1):
        if k % 20 == 0:
            print(f"  k={k}/{k_max}")
            
        stats = simulate_mc_with_model(k, model_config, mc_trials, parallel_factor, seed + k)
        
        # Calculate per-inference costs and times
        cost_per_inference = stats["cost"]["mean"] / k
        time_per_inference = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        
        # Check per-inference constraints
        if (cost_per_inference <= C_max_per_inference and 
            time_per_inference <= T_max_per_inference and 
            stats["acc"]["mean"] >= acc_min):
            
            if stats["acc"]["mean"] > best_acc:
                best_acc = stats["acc"]["mean"]
                best = {
                    "k": k,
                    "total_cost": stats["cost"]["mean"],
                    "total_time": stats["time"]["mean"],
                    "cost_per_inference": cost_per_inference,
                    "time_per_inference": time_per_inference,
                    "accuracy": stats["acc"]["mean"],
                    "stats": stats,
                    "model": model_config.name
                }

    return best


def find_maximum_cube_solution_mc(model_config, C_max_per_inference, T_max_per_inference, acc_min=0.0, k_max=200,
                                 mc_trials=500, parallel_factor=8, seed=42):
    """Find cube-optimal solution using per-inference constraints."""
    print(f"Finding cube-optimal solution with per-inference constraints...")
    
    best = None
    best_vol = -1
    
    for k in range(1, k_max + 1):
        if k % 20 == 0:
            print(f"  k={k}/{k_max}")
            
        stats = simulate_mc_with_model(k, model_config, mc_trials, parallel_factor, seed + k)
        
        # Calculate per-inference metrics
        cost_per_inference = stats["cost"]["mean"] / k
        time_per_inference = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        acc_mean = stats["acc"]["mean"]
        
        # Check per-inference constraints
        if (cost_per_inference <= C_max_per_inference and 
            time_per_inference <= T_max_per_inference and 
            acc_mean >= acc_min):
            
            # Cube volume based on per-inference goodness
            gC = max(0.0, 1.0 - cost_per_inference / C_max_per_inference)
            gT = max(0.0, 1.0 - time_per_inference / T_max_per_inference)
            gA = acc_mean
            vol = gC * gT * gA
            
            if vol > best_vol:
                best_vol = vol
                best = {
                    "k": k,
                    "total_cost": stats["cost"]["mean"],
                    "total_time": stats["time"]["mean"],
                    "cost_per_inference": cost_per_inference,
                    "time_per_inference": time_per_inference,
                    "accuracy": acc_mean,
                    "cube_volume": vol,
                    "stats": stats,
                    "model": model_config.name
                }

    return best


In [59]:
# ===============================
# PARETO, UTOPIA, KNEE, CUBE
# ===============================

def is_dominated(point1, point2):
    # points: (C, T, A)
    C1, T1, A1 = point1
    C2, T2, A2 = point2
    non_worse = (C2 <= C1) and (T2 <= T1) and (A2 >= A1)
    strictly_better = (C2 < C1) or (T2 < T1) or (A2 > A1)
    return non_worse and strictly_better


def find_pareto_frontier_mc(model_config, C_max_per_inference, T_max_per_inference, k_max=200, mc_trials=500, 
                           parallel_factor=8, seed=42):
    """Find Pareto frontier using per-inference constraints."""
    print(f"Finding Pareto frontier with per-inference constraints...")
    
    feasible_points = []
    
    for k in range(1, k_max + 1):
        if k % 20 == 0:
            print(f"  k={k}/{k_max}")
            
        stats = simulate_mc_with_model(k, model_config, mc_trials, parallel_factor, seed + k)
        
        # Calculate per-inference metrics
        cost_per_inference = stats["cost"]["mean"] / k
        time_per_inference = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        acc_mean = stats["acc"]["mean"]
        
        # Check per-inference constraints
        if (cost_per_inference <= C_max_per_inference and 
            time_per_inference <= T_max_per_inference):
            
            # Store both per-inference and total metrics
            feasible_points.append((
                k, 
                cost_per_inference, 
                time_per_inference, 
                acc_mean, 
                stats,
                stats["cost"]["mean"],  # total_cost
                stats["time"]["mean"]   # total_time
            ))

    if not feasible_points:
        return None

    # Pareto filtering based on per-inference values
    pareto = []
    for i, (k1, C1, T1, A1, stats1, tc1, tt1) in enumerate(feasible_points):
        dominated = False
        for j, (k2, C2, T2, A2, stats2, tc2, tt2) in enumerate(feasible_points):
            if i == j:
                continue
            if is_dominated((C1, T1, A1), (C2, T2, A2)):
                dominated = True
                break
        if not dominated:
            pareto.append((k1, C1, T1, A1, stats1, tc1, tt1))
    
    pareto.sort(key=lambda x: x[0])

    # Utopia-closest selection based on per-inference metrics
    def utopia_dist(C, T, A):
        return np.linalg.norm([C / C_max_per_inference, T / T_max_per_inference, 1 - A])

    best_item = min(pareto, key=lambda p: utopia_dist(p[1], p[2], p[3]))
    k_best, Cb, Tb, Ab, stats_best, tc_best, tt_best = best_item

    return {
        "k": k_best,
        "cost_per_inference": Cb,
        "time_per_inference": Tb,
        "total_cost": tc_best,
        "total_time": tt_best,
        "accuracy": Ab,
        "stats": stats_best,
        "pareto_points": [(k, C, T, A) for k, C, T, A, _, _, _ in pareto],
        "pareto_count": len(pareto),
        "feasible_points": len(feasible_points),
        "distance": utopia_dist(Cb, Tb, Ab),
        "model": model_config.name
    }


def find_knee_point(pareto_points, C_max_per_inference, T_max_per_inference):
    """Find knee point using per-inference normalization."""
    if pareto_points is None or len(pareto_points) < 3:
        return None
    
    # Normalize based on per-inference constraints
    P = np.array([[C / C_max_per_inference, T / T_max_per_inference, A] for _, C, T, A in pareto_points])
    start = P[0]
    end = P[-1]
    v = end - start
    vn = np.linalg.norm(v)
    if vn == 0:
        return pareto_points[len(pareto_points)//2]

    v_unit = v / vn
    max_d = -1
    max_i = 0
    for i in range(1, len(P)-1):
        w = P[i] - start
        proj = np.dot(w, v_unit) * v_unit
        perp = w - proj
        d = np.linalg.norm(perp)
        if d > max_d:
            max_d = d
            max_i = i
    return pareto_points[max_i]


# ===============================
# UPDATED COMPARISON WRAPPER
# ===============================

def compare_methods_mc(model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max=200, mc_trials=500, 
                      parallel_factor=8, seed=42):
    """Compare optimization methods using per-inference constraints."""
    print(f"\n🔄 Running Monte Carlo comparison for {model_config.name}")
    print(f"   Per-inference limits: C=${C_max_per_inference:.4f}, T={T_max_per_inference:.3f}s")
    print(f"   MC trials: {mc_trials}, k_max: {k_max}, P: {parallel_factor}")
    print("="*70)
    
    acc_res = find_accuracy_optimal_mc(model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max,
                                      mc_trials, parallel_factor, seed)
    
    cube_res = find_maximum_cube_solution_mc(model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max,
                                            mc_trials, parallel_factor, seed)
    
    pareto_res = find_pareto_frontier_mc(model_config, C_max_per_inference, T_max_per_inference, k_max,
                                        mc_trials, parallel_factor, seed)
    
    knee_res = None
    if pareto_res and pareto_res.get("pareto_points"):
        kp = find_knee_point(pareto_res["pareto_points"], C_max_per_inference, T_max_per_inference)
        if kp:
            k_knee, Ck, Tk, Ak = kp
            # Find stats for knee point
            knee_stats = simulate_mc_with_model(k_knee, model_config, mc_trials, 
                                               parallel_factor, seed + k_knee)
            knee_res = {
                "k": k_knee, 
                "cost_per_inference": Ck,
                "time_per_inference": Tk,
                "total_cost": knee_stats["cost"]["mean"],
                "total_time": knee_stats["time"]["mean"],
                "accuracy": Ak,
                "stats": knee_stats,
                "type": "knee_point"
            }
    
    return acc_res, cube_res, pareto_res, knee_res

# Viz 1


In [60]:
# ===============================
# UPDATED INTERACTIVE VISUALIZATION
# ===============================

def update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min, k_max=200, 
                     mc_trials=300, parallel_factor=None, seed=42):
    """Updated visualization using per-inference constraints."""
    model_config = MODEL_CONFIGS[selected_model]
    if parallel_factor is None:
        parallel_factor = model_config.default_parallel

    print(f"🤖 {model_config.name} | P={parallel_factor} | MC={mc_trials}")
    print(f"   Costs: ${model_config.c_in*1e6:.2f}/M in, ${model_config.c_out*1e6:.2f}/M out")
    print(f"   Times: {model_config.t_in*1e6:.0f}μs/in tok, {model_config.t_out*1e6:.0f}μs/out tok")
    print(f"   ACC ~ N({model_config.acc_mean:.3f}, {model_config.acc_std:.3f})")
    print(f"   Per-inference limits: C=${C_max_per_inference:.4f}, T={T_max_per_inference:.3f}s, ACC_min={acc_min:.3f}")

    # Generate response curves using Monte Carlo
    print("\n📊 Generating response curves...")
    ks = np.arange(1, min(k_max + 1, 51))  # Limit for visualization performance
    Cs_mean, Cs_std, Ts_mean, Ts_std, As_mean, As_std = [], [], [], [], [], []
    Cs_per_inf, Ts_per_inf = [], []  # Per-inference metrics
    
    for i, k in enumerate(ks):
        if i % 10 == 0:
            print(f"   k={k}/{len(ks)}")
        stats = simulate_mc_with_model(k, model_config, mc_trials//2, parallel_factor, seed + k)
        
        # Total metrics
        Cs_mean.append(stats["cost"]["mean"])
        Cs_std.append(stats["cost"]["std"])
        Ts_mean.append(stats["time"]["mean"])
        Ts_std.append(stats["time"]["std"])
        As_mean.append(stats["acc"]["mean"])
        As_std.append(stats["acc"]["std"])
        
        # Per-inference metrics
        cost_per_inf = stats["cost"]["mean"] / k
        time_per_inf = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        Cs_per_inf.append(cost_per_inf)
        Ts_per_inf.append(time_per_inf)

    Cs_mean, Ts_mean, As_mean = np.array(Cs_mean), np.array(Ts_mean), np.array(As_mean)
    Cs_std, Ts_std, As_std = np.array(Cs_std), np.array(Ts_std), np.array(As_std)
    Cs_per_inf, Ts_per_inf = np.array(Cs_per_inf), np.array(Ts_per_inf)

    # Run optimization methods with per-inference constraints
    acc_res, cube_res, pareto_res, knee_res = compare_methods_mc(
        model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max, mc_trials, parallel_factor, seed
    )

    # Visualization with per-inference feasibility
    fig = plt.figure(figsize=(18, 12))
    
    # 3D Plot using total costs/times but per-inference feasibility
    ax1 = fig.add_subplot(221, projection="3d")
    feas = (Cs_per_inf <= C_max_per_inference) & (Ts_per_inf <= T_max_per_inference) & (As_mean >= acc_min)
    
    # Cube volume based on per-inference goodness
    gC = np.clip(1 - Cs_per_inf / C_max_per_inference, 0, 1)
    gT = np.clip(1 - Ts_per_inf / T_max_per_inference, 0, 1)
    gA = np.clip(As_mean, 0, 1)
    cube = gC * gT * gA
    
    scatter = ax1.scatter(Cs_mean, Ts_mean, As_mean, c=cube, cmap="plasma", s=30, alpha=0.8)
    ax1.plot(Cs_mean, Ts_mean, As_mean, color="green", lw=1, alpha=0.5, label="MC Trajectory")
    cbar = plt.colorbar(scatter, ax=ax1, shrink=0.6, pad=0.1)
    cbar.set_label("Cube Volume (per-inf)")

    # Add constraint planes - these are now derived from per-inference limits
    max_feasible_k = len(ks)
    Cg, Tg = np.meshgrid(
        np.linspace(0, C_max_per_inference * max_feasible_k, 10), 
        np.linspace(0, T_max_per_inference * max_feasible_k, 10)
    )
    Ag = np.full_like(Cg, acc_min)
    ax1.plot_surface(Cg, Tg, Ag, color="green", alpha=0.15, label="ACC constraint")

    # Mark optimal solutions using total costs/times for visualization
    def mark_mc(ax, res, color, marker, label):
        if res:
            total_cost = res.get("total_cost", res.get("cost", 0))
            total_time = res.get("total_time", res.get("time", 0))
            ax.scatter(total_cost, total_time, res["accuracy"], 
                      color=color, s=140, edgecolors="black", marker=marker, 
                      linewidth=2, label=f"{label} k={res['k']}")

    mark_mc(ax1, acc_res, "gold", "o", "Accuracy-opt")
    mark_mc(ax1, cube_res, "orange", "^", "Cube-opt")
    mark_mc(ax1, pareto_res, "red", "D", "Utopia")
    mark_mc(ax1, knee_res, "purple", "s", "Knee")

    ax1.set_xlabel("Total Cost ($)")
    ax1.set_ylabel("Total Time (s)")
    ax1.set_zlabel("Accuracy")
    ax1.set_title(f"3D MC (Per-Inference Constraints): {model_config.name}")
    ax1.legend()

    # 2D plots with per-inference constraint lines
    ax2 = fig.add_subplot(222)
    ax2.errorbar(ks, As_mean, yerr=As_std, lw=1.2, alpha=0.7, label="Accuracy ± σ")
    ax2.scatter(ks[feas], As_mean[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax2.axhline(acc_min, ls="--", color="red", label=f"ACC_min={acc_min:.2f}")
    if acc_res: ax2.scatter([acc_res["k"]], [acc_res["accuracy"]], s=100, c="gold", edgecolors="black")
    if cube_res: ax2.scatter([cube_res["k"]], [cube_res["accuracy"]], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax2.scatter([pareto_res["k"]], [pareto_res["accuracy"]], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax2.scatter([knee_res["k"]], [knee_res["accuracy"]], s=80, c="purple", edgecolors="black", marker="s")
    ax2.set_xlabel("k")
    ax2.set_ylabel("Accuracy")
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_title("Accuracy vs k (Per-Inf Constraints)")

    # ax3 = fig.add_subplot(223)
    # ax3.errorbar(ks, Cs_per_inf, lw=1.2, alpha=0.7, label="Cost per inference")
    # ax3.scatter(ks[feas], Cs_per_inf[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    # ax3.axhline(C_max_per_inference, ls="--", color="red", label=f"C_max/inf=${C_max_per_inference:.4f}")
    # if acc_res: ax3.scatter([acc_res["k"]], [acc_res.get("cost_per_inference", 0)], s=100, c="gold", edgecolors="black")
    # if cube_res: ax3.scatter([cube_res["k"]], [cube_res.get("cost_per_inference", 0)], s=80, c="orange", edgecolors="black", marker="^")
    # if pareto_res: ax3.scatter([pareto_res["k"]], [pareto_res.get("cost_per_inference", 0)], s=80, c="red", edgecolors="black", marker="D")
    # if knee_res: ax3.scatter([knee_res["k"]], [knee_res.get("cost_per_inference", 0)], s=80, c="purple", edgecolors="black", marker="s")
    # ax3.set_xlabel("k")
    # ax3.set_ylabel("Cost per inference ($)")
    # ax3.grid(True, alpha=0.3)
    # ax3.legend()
    # ax3.set_title("Cost per inference vs k")

    ax4 = fig.add_subplot(224)
    ax4.errorbar(ks, Ts_per_inf, lw=1.2, alpha=0.7, label="Time per inference")
    ax4.scatter(ks[feas], Ts_per_inf[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax4.axhline(T_max_per_inference, ls="--", color="blue", label=f"T_max/inf={T_max_per_inference:.3f}s")
    if acc_res: ax4.scatter([acc_res["k"]], [acc_res.get("time_per_inference", 0)], s=100, c="gold", edgecolors="black")
    if cube_res: ax4.scatter([cube_res["k"]], [cube_res.get("time_per_inference", 0)], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax4.scatter([pareto_res["k"]], [pareto_res.get("time_per_inference", 0)], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax4.scatter([knee_res["k"]], [knee_res.get("time_per_inference", 0)], s=80, c="purple", edgecolors="black", marker="s")
    ax4.set_xlabel("k")
    ax4.set_ylabel("Time per inference (s)")
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    ax4.set_title("Time per inference vs k")

    plt.tight_layout()
    plt.show()

    # Enhanced text summary with per-inference metrics
    print("\n📋 MONTE CARLO OPTIMIZATION RESULTS (Per-Inference Constraints)")
    print("="*70)
    
    if acc_res:
        stats = acc_res["stats"]
        print(f"🎯 Accuracy-Optimal: k={acc_res['k']}")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inference: ${acc_res['cost_per_inference']:.4f}, {acc_res['time_per_inference']:.3f}s")
        print(f"   Total: ${acc_res['total_cost']:.3f}, {acc_res['total_time']:.1f}s")
    
    if cube_res:
        stats = cube_res["stats"]
        print(f"\n🔶 Cube-Optimal: k={cube_res['k']} (vol={cube_res['cube_volume']:.3f})")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inference: ${cube_res['cost_per_inference']:.4f}, {cube_res['time_per_inference']:.3f}s")
        print(f"   Total: ${cube_res['total_cost']:.3f}, {cube_res['total_time']:.1f}s")
    
    if pareto_res:
        stats = pareto_res["stats"]
        print(f"\n🔴 Utopia-Closest: k={pareto_res['k']} (dist={pareto_res['distance']:.3f})")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inference: ${pareto_res['cost_per_inference']:.4f}, {pareto_res['time_per_inference']:.3f}s")
        print(f"   Total: ${pareto_res['total_cost']:.3f}, {pareto_res['total_time']:.1f}s")
        print(f"   Pareto points: {pareto_res['pareto_count']}/{pareto_res['feasible_points']}")
    
    if knee_res:
        stats = knee_res["stats"]
        print(f"\n🟣 Knee-Point: k={knee_res['k']}")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inference: ${knee_res['cost_per_inference']:.4f}, {knee_res['time_per_inference']:.3f}s")
        print(f"   Total: ${knee_res['total_cost']:.3f}, {knee_res['total_time']:.1f}s")


# ===============================
# UPDATED WIDGET WITH PER-INFERENCE CONSTRAINTS
# ===============================

# Update budget defaults to per-inference values
C_max_per_inference_default = 0.050  # $0.05 per inference 
T_max_per_inference_default = 5.0    # 5 seconds per inference

print("🚀 Monte Carlo Inference Scaling Optimization")
print("   Now with PER-INFERENCE constraints and proper MC averaging!")

widgets.interact(
    update_visuals_mc,
    selected_model=widgets.Dropdown(options=list(MODEL_CONFIGS.keys()),
                                    value='gpt5', description='Model'),
    C_max_per_inference=widgets.FloatSlider(min=0.001, max=0.200, step=0.001,
                              value=C_max_per_inference_default, description="Max Cost/Inf ($)"),
    T_max_per_inference=widgets.FloatSlider(min=0.5, max=30.0, step=0.1,
                              value=T_max_per_inference_default, description="Max Time/Inf (s)"),
    acc_min=widgets.FloatSlider(min=0.70, max=0.99, step=0.01,
                                value=0.83, description="Min ACC"),
    k_max=widgets.IntSlider(min=2, max=16*16, step=2, value=2**5, description="k_max"),
    mc_trials=widgets.IntSlider(min=100, max=500, step=10, value=100, description="MC Trials"),
    parallel_factor=widgets.IntSlider(min=1, max=256, step=1,
                                      value=MODEL_CONFIGS['gpt5'].default_parallel,
                                      description="Parallelism (P)"),
    # seed=widgets.IntText(value=42, description="Seed")
)

🚀 Monte Carlo Inference Scaling Optimization
   Now with PER-INFERENCE constraints and proper MC averaging!


interactive(children=(Dropdown(description='Model', options=('gpt5', 'gpt5-mini', 'gpt5-nano', 'nvidia-nemotro…

<function __main__.update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min, k_max=200, mc_trials=300, parallel_factor=None, seed=42)>

# viz 2

In [64]:
# ===============================
# UPDATED INTERACTIVE VISUALIZATION WITH COLORED CONSTRAINTS & BOLDED CUBE
# ===============================
# UPDATED INTERACTIVE VISUALIZATION WITH CONSTRAINT SURFACES & BOLDED CUBE

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

def update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min,
                     k_max=200, mc_trials=300, parallel_factor=None, seed=42):
    """Visualization with colored constraints + bolded feasible cube."""
    model_config = MODEL_CONFIGS[selected_model]
    if parallel_factor is None:
        parallel_factor = model_config.default_parallel

    print(f"🤖 {model_config.name} | P={parallel_factor} | MC={mc_trials}")
    print(f"   Costs: ${model_config.c_in*1e6:.2f}/M in, ${model_config.c_out*1e6:.2f}/M out")
    print(f"   Times: {model_config.t_in*1e6:.0f}μs/in tok, {model_config.t_out*1e6:.0f}μs/out tok")
    print(f"   ACC ~ N({model_config.acc_mean:.3f}, {model_config.acc_std:.3f})")
    print(f"   Per-inference limits: C=${C_max_per_inference:.4f}, T={T_max_per_inference:.3f}s, ACC_min={acc_min:.3f}")

    # Generate response curves using Monte Carlo
    print("\n📊 Generating response curves...")
    ks = np.arange(1, min(k_max + 1, 51))
    Cs_mean, Ts_mean, As_mean = [], [], []
    Cs_std, Ts_std, As_std = [], [], []
    Cs_per_inf, Ts_per_inf = [], []

    for i, k in enumerate(ks):
        if i % 10 == 0:
            print(f"   k={k}/{len(ks)}")
        stats = simulate_mc_with_model(k, model_config, mc_trials//2, parallel_factor, seed + k)

        Cs_mean.append(stats["cost"]["mean"])
        Cs_std.append(stats["cost"]["std"])
        Ts_mean.append(stats["time"]["mean"])
        Ts_std.append(stats["time"]["std"])
        As_mean.append(stats["acc"]["mean"])
        As_std.append(stats["acc"]["std"])

        # Per-inference metrics
        cost_per_inf = stats["cost"]["mean"] / k
        time_per_inf = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        Cs_per_inf.append(cost_per_inf)
        Ts_per_inf.append(time_per_inf)

    Cs_mean, Ts_mean, As_mean = np.array(Cs_mean), np.array(Ts_mean), np.array(As_mean)
    Cs_std, Ts_std, As_std = np.array(Cs_std), np.array(Ts_std), np.array(As_std)
    Cs_per_inf, Ts_per_inf = np.array(Cs_per_inf), np.array(Ts_per_inf)

    # Run optimization methods
    acc_res, cube_res, pareto_res, knee_res = compare_methods_mc(
        model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max, mc_trials, parallel_factor, seed
    )

    # === Visualization ===
    fig = plt.figure(figsize=(18, 12))
    ax1 = fig.add_subplot(221, projection="3d")

    feas = (Cs_per_inf <= C_max_per_inference) & (Ts_per_inf <= T_max_per_inference) & (As_mean >= acc_min)

    # Cube-volume color
    gC = np.clip(1 - Cs_per_inf / C_max_per_inference, 0, 1)
    gT = np.clip(1 - Ts_per_inf / T_max_per_inference, 0, 1)
    gA = np.clip(As_mean, 0, 1)
    cube = gC * gT * gA

    # Scatter MC points
    sc = ax1.scatter(Cs_mean, Ts_mean, As_mean, c=cube, cmap="plasma", s=40, alpha=0.8)
    ax1.plot(Cs_mean, Ts_mean, As_mean, color="gray", lw=1, alpha=0.5, label="MC trajectory")
    plt.colorbar(sc, ax=ax1, shrink=0.6, pad=0.1, label="Cube Volume")

    # === Add feasible cube with colored constraint planes ===
    Cmax = C_max_per_inference * len(ks)
    Tmax = T_max_per_inference * len(ks)
    Amin = acc_min
    Cg, Tg = np.meshgrid(np.linspace(0, Cmax, 20), np.linspace(0, Tmax, 20))

    # Constraint planes
    # Cost plane (red)
    ax1.plot_surface(np.full_like(Cg, Cmax), Tg, np.full_like(Cg, Amin + (1 - Amin)),
                     color="tomato", alpha=0.20)

    # Time plane (blue)
    ax1.plot_surface(Cg, np.full_like(Tg, Tmax), np.full_like(Cg, Amin + (1 - Amin)),
                     color="royalblue", alpha=0.20)

    # Accuracy plane (green)
    ax1.plot_surface(Cg, Tg, np.full_like(Cg, Amin),
                     color="limegreen", alpha=0.25)

    # Cube faces (transparent fill)
    verts = [
        [0, 0, Amin], [Cmax, 0, Amin], [Cmax, Tmax, Amin], [0, Tmax, Amin],
        [0, 0, 1], [Cmax, 0, 1], [Cmax, Tmax, 1], [0, Tmax, 1]
    ]
    faces = [
        [verts[j] for j in [0,1,2,3]],
        [verts[j] for j in [4,5,6,7]],
        [verts[j] for j in [0,1,5,4]],
        [verts[j] for j in [2,3,7,6]],
        [verts[j] for j in [1,2,6,5]],
        [verts[j] for j in [4,7,3,0]]
    ]
    ax1.add_collection3d(Poly3DCollection(faces, color="lightgreen", alpha=0.05))

    # Cube edges (bold)
    edge_color = "black"
    edge_thick = 1.8
    for X in [0, Cmax]:
        for Y in [0, Tmax]:
            ax1.plot([X, X], [Y, Y], [Amin, 1], color=edge_color, lw=edge_thick, alpha=0.9)
    for X in [0, Cmax]:
        for Z in [Amin, 1]:
            ax1.plot([X, X], [0, Tmax], [Z, Z], color=edge_color, lw=edge_thick, alpha=0.9)
    for Y in [0, Tmax]:
        for Z in [Amin, 1]:
            ax1.plot([0, Cmax], [Y, Y], [Z, Z], color=edge_color, lw=edge_thick, alpha=0.9)

    # Mark optimal points
    def mark_mc(ax, res, color, marker, label):
        if res:
            ax.scatter(res.get("total_cost", res.get("cost", 0)),
                       res.get("total_time", res.get("time", 0)),
                       res["accuracy"], color=color, edgecolors="black",
                       s=140, marker=marker, linewidth=1.5,
                       label=f"{label} k={res['k']}")

    mark_mc(ax1, acc_res, "gold", "o", "Accuracy-opt")
    mark_mc(ax1, cube_res, "orange", "^", "Cube-opt")
    mark_mc(ax1, pareto_res, "red", "D", "Utopia")
    mark_mc(ax1, knee_res, "purple", "s", "Knee")

    ax1.set_xlabel("Cost ($)")
    ax1.set_ylabel("Time (/s)")
    ax1.set_zlabel("Performance (ACC)")
    ax1.set_xlim(0, Cmax * 1.05)
    ax1.set_ylim(0, Tmax * 1.05)
    ax1.set_zlim(Amin - 0.02, 1.0)
    ax1.set_title(f"3D Feasible Cube — {model_config.name}")
    ax1.legend(loc="upper left", fontsize=8)

    # -------------------------
    # 2D Accuracy vs k
    # -------------------------
    ax2 = fig.add_subplot(222)
    ax2.errorbar(ks, As_mean, yerr=As_std, lw=1.2, alpha=0.7, label="Accuracy ± σ")
    ax2.scatter(ks[feas], As_mean[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax2.axhline(acc_min, ls="--", color="red", label=f"ACC_min={acc_min:.2f}")
    if acc_res: ax2.scatter([acc_res["k"]], [acc_res["accuracy"]], s=100, c="gold", edgecolors="black")
    if cube_res: ax2.scatter([cube_res["k"]], [cube_res["accuracy"]], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax2.scatter([pareto_res["k"]], [pareto_res["accuracy"]], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax2.scatter([knee_res["k"]], [knee_res["accuracy"]], s=80, c="purple", edgecolors="black", marker="s")
    ax2.set_xlabel("k")
    ax2.set_ylabel("Accuracy")
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_title("Accuracy vs k (Per-Inf Constraints)")

    # -------------------------
    # 2D Time per inference
    # -------------------------
    ax4 = fig.add_subplot(224)
    ax4.errorbar(ks, Ts_per_inf, lw=1.2, alpha=0.7, label="Time per inference")
    ax4.scatter(ks[feas], Ts_per_inf[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax4.axhline(T_max_per_inference, ls="--", color="blue", label=f"T_max/inf={T_max_per_inference:.3f}s")
    if acc_res: ax4.scatter([acc_res["k"]], [acc_res.get("time_per_inference", 0)], s=100, c="gold", edgecolors="black")
    if cube_res: ax4.scatter([cube_res["k"]], [cube_res.get("time_per_inference", 0)], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax4.scatter([pareto_res["k"]], [pareto_res.get("time_per_inference", 0)], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax4.scatter([knee_res["k"]], [knee_res.get("time_per_inference", 0)], s=80, c="purple", edgecolors="black", marker="s")
    ax4.set_xlabel("k")
    ax4.set_ylabel("Time per inference (s)")
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    ax4.set_title("Time per inference vs k")

    plt.tight_layout()
    plt.show()

    # -------------------------
    # Text Summary
    # -------------------------
    print("\n📋 MONTE CARLO OPTIMIZATION RESULTS (Per-Inference Constraints)")
    print("="*70)
    def print_res(label, emoji, res, extra=""):
        if not res: return
        stats = res["stats"]
        print(f"\n{emoji} {label}: k={res['k']} {extra}")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inf: ${res['cost_per_inference']:.4f}, {res['time_per_inference']:.3f}s")
        print(f"   Total: ${res['total_cost']:.3f}, {res['total_time']:.1f}s")

    print_res("Accuracy-Optimal", "🎯", acc_res)
    print_res("Cube-Optimal", "🔶", cube_res, f"(vol={cube_res.get('cube_volume', 0):.3f})")
    print_res("Utopia-Closest", "🔴", pareto_res, f"(dist={pareto_res.get('distance', 0):.3f})")
    print_res("Knee-Point", "🟣", knee_res)


# ===============================
# INTERACTIVE WIDGET SETUP
# ===============================
C_max_per_inference_default = 0.050
T_max_per_inference_default = 5.0

print("🚀 Monte Carlo Inference Scaling Optimization")
print("   With PER-INFERENCE constraints and colored 3D FEASIBLE CUBE visualization!")

widgets.interact(
    update_visuals_mc,
    selected_model=widgets.Dropdown(options=list(MODEL_CONFIGS.keys()),
                                    value='gpt5', description='Model'),
    C_max_per_inference=widgets.FloatSlider(min=0.001, max=0.900, step=0.001,
                              value=C_max_per_inference_default, description="Max Cost/Inf ($)"),
    T_max_per_inference=widgets.FloatSlider(min=10, max=6000.0, step=0.1,
                              value=T_max_per_inference_default, description="Max Time/Inf (s)"),
    acc_min=widgets.FloatSlider(min=0.70, max=0.99, step=0.01,
                                value=0.83, description="Min ACC"),
    k_max=widgets.IntSlider(min=2, max=16*16, step=2, value=2**5, description="k_max"),
    mc_trials=widgets.IntSlider(min=300, max=500, step=10, value=100, description="MC Trials"),
    parallel_factor=widgets.IntSlider(min=1, max=256, step=1,
                                      value=MODEL_CONFIGS['gpt5'].default_parallel,
                                      description="Parallelism (P)")
)


🚀 Monte Carlo Inference Scaling Optimization
   With PER-INFERENCE constraints and colored 3D FEASIBLE CUBE visualization!


interactive(children=(Dropdown(description='Model', options=('gpt5', 'gpt5-mini', 'gpt5-nano', 'nvidia-nemotro…

<function __main__.update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min, k_max=200, mc_trials=300, parallel_factor=None, seed=42)>

# viz 3

In [65]:
# ===============================
# UPDATED VISUALIZATION — ONE FIGURE PER PLOT
# ===============================

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

def update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min,
                     k_max=200, mc_trials=300, parallel_factor=None, seed=42):
    """Generate three separate figures instead of one combined image."""
    model_config = MODEL_CONFIGS[selected_model]
    if parallel_factor is None:
        parallel_factor = model_config.default_parallel

    print(f"🤖 {model_config.name} | P={parallel_factor} | MC={mc_trials}")
    print(f"   Costs: ${model_config.c_in*1e6:.2f}/M in, ${model_config.c_out*1e6:.2f}/M out")
    print(f"   Times: {model_config.t_in*1e6:.0f}μs/in tok, {model_config.t_out*1e6:.0f}μs/out tok")
    print(f"   ACC ~ N({model_config.acc_mean:.3f}, {model_config.acc_std:.3f})")
    print(f"   Per-inference limits: C=${C_max_per_inference:.4f}, T={T_max_per_inference:.3f}s, ACC_min={acc_min:.3f}")

    # Monte Carlo simulations
    print("\n📊 Generating response curves...")
    ks = np.arange(1, min(k_max + 1, 51))
    Cs_mean, Ts_mean, As_mean = [], [], []
    Cs_std, Ts_std, As_std = [], [], []
    Cs_per_inf, Ts_per_inf = [], []

    for i, k in enumerate(ks):
        if i % 10 == 0:
            print(f"   k={k}/{len(ks)}")
        stats = simulate_mc_with_model(k, model_config, mc_trials//2, parallel_factor, seed + k)

        Cs_mean.append(stats["cost"]["mean"])
        Cs_std.append(stats["cost"]["std"])
        Ts_mean.append(stats["time"]["mean"])
        Ts_std.append(stats["time"]["std"])
        As_mean.append(stats["acc"]["mean"])
        As_std.append(stats["acc"]["std"])

        cost_per_inf = stats["cost"]["mean"] / k
        time_per_inf = stats["time"]["mean"] / k if parallel_factor == 1 else stats["time"]["mean"] / min(k, parallel_factor)
        Cs_per_inf.append(cost_per_inf)
        Ts_per_inf.append(time_per_inf)

    Cs_mean, Ts_mean, As_mean = np.array(Cs_mean), np.array(Ts_mean), np.array(As_mean)
    Cs_std, Ts_std, As_std = np.array(Cs_std), np.array(Ts_std), np.array(As_std)
    Cs_per_inf, Ts_per_inf = np.array(Cs_per_inf), np.array(Ts_per_inf)

    # Run optimization methods
    acc_res, cube_res, pareto_res, knee_res = compare_methods_mc(
        model_config, C_max_per_inference, T_max_per_inference, acc_min, k_max, mc_trials, parallel_factor, seed
    )

    # =====================================================
    # FIGURE 1 — 3D Feasible Cube
    # =====================================================
    fig1 = plt.figure(figsize=(8, 6))
    ax1 = fig1.add_subplot(111, projection="3d")

    feas = (Cs_per_inf <= C_max_per_inference) & (Ts_per_inf <= T_max_per_inference) & (As_mean >= acc_min)
    gC = np.clip(1 - Cs_per_inf / C_max_per_inference, 0, 1)
    gT = np.clip(1 - Ts_per_inf / T_max_per_inference, 0, 1)
    gA = np.clip(As_mean, 0, 1)
    cube = gC * gT * gA

    sc = ax1.scatter(Cs_mean, Ts_mean, As_mean, c=cube, cmap="plasma", s=40, alpha=0.8)
    ax1.plot(Cs_mean, Ts_mean, As_mean, color="gray", lw=1, alpha=0.5, label="MC trajectory")
    plt.colorbar(sc, ax=ax1, shrink=0.6, pad=0.1, label="Cube Volume")

    Cmax = C_max_per_inference * len(ks)
    Tmax = T_max_per_inference * len(ks)
    Amin = acc_min
    Cg, Tg = np.meshgrid(np.linspace(0, Cmax, 20), np.linspace(0, Tmax, 20))

    # Constraint planes
    ax1.plot_surface(np.full_like(Cg, Cmax), Tg, np.full_like(Cg, Amin + (1 - Amin)),
                     color="tomato", alpha=0.20)
    ax1.plot_surface(Cg, np.full_like(Tg, Tmax), np.full_like(Cg, Amin + (1 - Amin)),
                     color="royalblue", alpha=0.20)
    ax1.plot_surface(Cg, Tg, np.full_like(Cg, Amin),
                     color="limegreen", alpha=0.25)

    # Cube faces and edges
    verts = [
        [0, 0, Amin], [Cmax, 0, Amin], [Cmax, Tmax, Amin], [0, Tmax, Amin],
        [0, 0, 1], [Cmax, 0, 1], [Cmax, Tmax, 1], [0, Tmax, 1]
    ]
    faces = [
        [verts[j] for j in [0,1,2,3]],
        [verts[j] for j in [4,5,6,7]],
        [verts[j] for j in [0,1,5,4]],
        [verts[j] for j in [2,3,7,6]],
        [verts[j] for j in [1,2,6,5]],
        [verts[j] for j in [4,7,3,0]]
    ]
    ax1.add_collection3d(Poly3DCollection(faces, color="lightgreen", alpha=0.05))

    edge_color, edge_thick = "black", 1.8
    for X in [0, Cmax]:
        for Y in [0, Tmax]:
            ax1.plot([X, X], [Y, Y], [Amin, 1], color=edge_color, lw=edge_thick, alpha=0.9)
    for X in [0, Cmax]:
        for Z in [Amin, 1]:
            ax1.plot([X, X], [0, Tmax], [Z, Z], color=edge_color, lw=edge_thick, alpha=0.9)
    for Y in [0, Tmax]:
        for Z in [Amin, 1]:
            ax1.plot([0, Cmax], [Y, Y], [Z, Z], color=edge_color, lw=edge_thick, alpha=0.9)

    def mark_mc(ax, res, color, marker, label):
        if res:
            ax.scatter(res.get("total_cost", res.get("cost", 0)),
                       res.get("total_time", res.get("time", 0)),
                       res["accuracy"], color=color, edgecolors="black",
                       s=140, marker=marker, linewidth=1.5,
                       label=f"{label} k={res['k']}")

    mark_mc(ax1, acc_res, "gold", "o", "Accuracy-opt")
    mark_mc(ax1, cube_res, "orange", "^", "Cube-opt")
    mark_mc(ax1, pareto_res, "red", "D", "Utopia")
    mark_mc(ax1, knee_res, "purple", "s", "Knee")

    ax1.set_xlabel("Cost ($)")
    ax1.set_ylabel("Time (s)")
    ax1.set_zlabel("Accuracy")
    ax1.set_title(f"3D Feasible Cube — {model_config.name}")
    ax1.legend(loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.show()
    # fig1.savefig(f"{model_config.name}_3D_cube.pdf", bbox_inches="tight")

    # =====================================================
    # FIGURE 2 — Accuracy vs k
    # =====================================================
    fig2, ax2 = plt.subplots(figsize=(6, 4))
    ax2.errorbar(ks, As_mean, yerr=As_std, lw=1.2, alpha=0.7, label="Accuracy ± σ")
    ax2.scatter(ks[feas], As_mean[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax2.axhline(acc_min, ls="--", color="red", label=f"ACC_min={acc_min:.2f}")
    if acc_res: ax2.scatter([acc_res["k"]], [acc_res["accuracy"]], s=100, c="gold", edgecolors="black")
    if cube_res: ax2.scatter([cube_res["k"]], [cube_res["accuracy"]], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax2.scatter([pareto_res["k"]], [pareto_res["accuracy"]], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax2.scatter([knee_res["k"]], [knee_res["accuracy"]], s=80, c="purple", edgecolors="black", marker="s")
    ax2.set_xlabel("k")
    ax2.set_ylabel("Accuracy")
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_title("Accuracy vs k (Per-Inf Constraints)")
    plt.tight_layout()
    plt.show()
    # fig2.savefig(f"{model_config.name}_Accuracy_vs_k.pdf", bbox_inches="tight")

    # =====================================================
    # FIGURE 3 — Time per Inference vs k
    # =====================================================
    fig3, ax3 = plt.subplots(figsize=(6, 4))
    ax3.errorbar(ks, Ts_per_inf, lw=1.2, alpha=0.7, label="Time per inference")
    ax3.scatter(ks[feas], Ts_per_inf[feas], s=12, color="lightgreen", alpha=0.7, label="Feasible")
    ax3.axhline(T_max_per_inference, ls="--", color="blue", label=f"T_max/inf={T_max_per_inference:.3f}s")
    if acc_res: ax3.scatter([acc_res["k"]], [acc_res.get("time_per_inference", 0)], s=100, c="gold", edgecolors="black")
    if cube_res: ax3.scatter([cube_res["k"]], [cube_res.get("time_per_inference", 0)], s=80, c="orange", edgecolors="black", marker="^")
    if pareto_res: ax3.scatter([pareto_res["k"]], [pareto_res.get("time_per_inference", 0)], s=80, c="red", edgecolors="black", marker="D")
    if knee_res: ax3.scatter([knee_res["k"]], [knee_res.get("time_per_inference", 0)], s=80, c="purple", edgecolors="black", marker="s")
    ax3.set_xlabel("k")
    ax3.set_ylabel("Time per inference (s)")
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    ax3.set_title("Time per inference vs k")
    plt.tight_layout()
    plt.show()
    # fig3.savefig(f"{model_config.name}_Time_vs_k.pdf", bbox_inches="tight")

    # =====================================================
    # Text Summary
    # =====================================================
    print("\n📋 MONTE CARLO OPTIMIZATION RESULTS (Per-Inference Constraints)")
    print("="*70)
    def print_res(label, emoji, res, extra=""):
        if not res: return
        stats = res["stats"]
        print(f"\n{emoji} {label}: k={res['k']} {extra}")
        print(f"   ACC: {stats['acc']['mean']:.3f} ± {stats['acc']['std']:.3f}")
        print(f"   Per-inf: ${res['cost_per_inference']:.4f}, {res['time_per_inference']:.3f}s")
        print(f"   Total: ${res['total_cost']:.3f}, {res['total_time']:.1f}s")

    print_res("Accuracy-Optimal", "🎯", acc_res)
    print_res("Cube-Optimal", "🔶", cube_res, f"(vol={cube_res.get('cube_volume', 0):.3f})")
    print_res("Utopia-Closest", "🔴", pareto_res, f"(dist={pareto_res.get('distance', 0):.3f})")
    print_res("Knee-Point", "🟣", knee_res)


# ===============================
# INTERACTIVE WIDGET SETUP
# ===============================
C_max_per_inference_default = 0.050
T_max_per_inference_default = 5.0

print("🚀 Monte Carlo Inference Scaling Optimization — One Figure per Plot!")

widgets.interact(
    update_visuals_mc,
    selected_model=widgets.Dropdown(options=list(MODEL_CONFIGS.keys()),
                                    value='gpt5', description='Model'),
    C_max_per_inference=widgets.FloatSlider(min=0.001, max=0.900, step=0.001,
                              value=C_max_per_inference_default, description="Max Cost/Inf ($)"),
    T_max_per_inference=widgets.FloatSlider(min=10, max=6000.0, step=0.1,
                              value=T_max_per_inference_default, description="Max Time/Inf (s)"),
    acc_min=widgets.FloatSlider(min=0.70, max=0.99, step=0.01,
                                value=0.83, description="Min ACC"),
    k_max=widgets.IntSlider(min=2, max=16*16, step=2, value=2**5, description="k_max"),
    mc_trials=widgets.IntSlider(min=300, max=500, step=10, value=100, description="MC Trials"),
    parallel_factor=widgets.IntSlider(min=1, max=256, step=1,
                                      value=MODEL_CONFIGS['gpt5'].default_parallel,
                                      description="Parallelism (P)")
)


🚀 Monte Carlo Inference Scaling Optimization — One Figure per Plot!


interactive(children=(Dropdown(description='Model', options=('gpt5', 'gpt5-mini', 'gpt5-nano', 'nvidia-nemotro…

<function __main__.update_visuals_mc(selected_model, C_max_per_inference, T_max_per_inference, acc_min, k_max=200, mc_trials=300, parallel_factor=None, seed=42)>